## Update features api rest library: requests


#### imports

In [4]:
import pandas as pd
import numpy as np
from datetime import datetime
import math
import dateparser
import datetime
import requests
from arcgis.gis import GIS
gis = GIS("home")
from arcgis.features import FeatureLayerCollection

In [2]:
!pip install dateparser

     |████████████████████████████████| 358kB 9.9MB/s eta 0:00:01
     |████████████████████████████████| 686kB 39.0MB/s eta 0:00:01


## update

In [5]:
def pull_data():
    dfo=pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
    df=dfo
    df['Province/State'].fillna(value='',inplace=True)
    df['site']=df['Country/Region']+'-'+df['Province/State']
    #clean the countries with little data 
    df=df[df[list(df)[-2]]!=0]
    df=df[df[list(df)[-3]]!=0]
    df=df[df[list(df)[-4]]!=0]
    def clean(name):
        if name[-1]=='-':
            name=name[:-1]
        else:
            name=name
        return name
    #clean site name to do the filters
    df['site']=df['site'].apply(lambda x: clean(x))
    sites=list(df['site'].unique())
    #reshape the data to add a predicted feature in the next step
    df0=df.groupby(['Country/Region','Province/State','site','Lat','Long']).sum().stack().reset_index()
    df0.rename(columns={'level_5':'date',0:'accumulated'}, inplace=True)
    df0['date']=df0['date'].apply(lambda x: dateparser.parse(x))#, date_formats=['%Y %d %B'], languages=['es']))
    return sites,df0
one_day=datetime.timedelta(days=1)
yesterday=pd.Timestamp(datetime.date.today()-one_day)
#prepare and calculate de status of the growth
def prepare(df0,site):
    df_data=df0[df0['site']==site]
    df_data=df_data[df_data.accumulated!=0]
    df_data['days']=np.arange(len(df_data.index))+1
    df_data=df_data[['site','date','accumulated','days']]
    y=np.array(df_data.accumulated)
    df_data=df_data[df_data['date']>yesterday-one_day]
    df_data['type']='data'
    if y[-1]<=y[-2]:
        df_data['growth']='Estabilizado'
    else:
        df_data['growth']='Incrementando'
    return df_data
# run prepare and convert the data to a json to do the request
def preparejson(sites,df0):
    add_df=pd.DataFrame()
    for s in sites:
        dfi=prepare(df0,s)
        add_df=pd.concat([add_df, dfi])
    add_df=add_df.reset_index().drop(columns='index')
    fivehours=datetime.timedelta(hours=5)
    add_df['date']=add_df['date']+fivehours
    lista = [('{attributes:'+add_df.loc[x].to_json()+'}') for x in list(add_df.index)]
    json0='['
    for i in lista:
        json0=json0+i+','
    json0=json0[:-1]+']'
    return json0
#run all and do the post
def update():
    sites,df0=pull_data()
    json0=preparejson(sites,df0)
    item_forecast = gis.content.get("service_id")
    collection = FeatureLayerCollection.fromitem(item_forecast)
    update_dict0 = {"capabilities": "Create,Delete,Query,Update,Editing"}
    collection.manager.update_definition(update_dict0)   
    url = "feature_Service_url/0/applyEdits"
    payload = 'f=json&adds={}'.format(json0)
    headers = {
      'Content-Type': 'application/x-www-form-urlencoded'
        }
    response = requests.request("POST", url, headers=headers, data = payload)
    update_dict1 = {"capabilities": "Query"}
    collection.manager.update_definition(update_dict1)
    print(response.text.encode('utf8'))
update()

/opt/conda/lib/python3.6/site-packages/tzlocal/unix.py:158: UserWarning: Can not find any timezone configuration, defaulting to UTC.
  warnings.warn('Can not find any timezone configuration, defaulting to UTC.')


b'{"addResults":[{"objectId":36145,"uniqueId":36145,"globalId":null,"success":true},{"objectId":36146,"uniqueId":36146,"globalId":null,"success":true},{"objectId":36147,"uniqueId":36147,"globalId":null,"success":true},{"objectId":36148,"uniqueId":36148,"globalId":null,"success":true},{"objectId":36149,"uniqueId":36149,"globalId":null,"success":true},{"objectId":36150,"uniqueId":36150,"globalId":null,"success":true},{"objectId":36151,"uniqueId":36151,"globalId":null,"success":true},{"objectId":36152,"uniqueId":36152,"globalId":null,"success":true},{"objectId":36153,"uniqueId":36153,"globalId":null,"success":true},{"objectId":36154,"uniqueId":36154,"globalId":null,"success":true},{"objectId":36155,"uniqueId":36155,"globalId":null,"success":true},{"objectId":36156,"uniqueId":36156,"globalId":null,"success":true},{"objectId":36157,"uniqueId":36157,"globalId":null,"success":true},{"objectId":36158,"uniqueId":36158,"globalId":null,"success":true},{"objectId":36159,"uniqueId":36159,"globalId"